# Parsing av vedtakstekster

Vedtakene i tekstkorpuset må parses, dvs. vi må identifisere strukturen i vedtakene, og dele opp vedtakene i deres ulike deler. Formålet er å analysere korpuset, og å legge til rette for å en variert testing opp mot språkmodellen. Eksempelvis ønsker vi å teste ut semantiske søk der vi bare søker i klagers anførsler. Da kan vi etterpå bruke nemndas vurdering fra de samme vedtakene som kontekst opp mot språkmodellen. Et annet bruksområde er å identifisere og analysere vedtak der nemnda deler seg i et flertall og et mindretall. Da er det viktig å først identifisere hvilken del av vedtaket som er nemndas vurdering. 


## Analyse av strukturen i vedtakstekstene
Det ser ut til at mange vedtak har samme struktur: sammendrag, dato, saksnr. og tjenesteyter, så klagers anførsler, tjenesteyters anførsler og til slutt nemndas bemerkninger. Før vi går i gang med selve parsingen, skal vi analysere strukturen vedtakene. Hvilke overskrifter bruker vedtakene, har vedtakene de sammen overkriftene? Hvilke stabile strukturer vi kan basere oss på når vi skal parse teksten i vedtakene?

Prinsippielt er det mulig tilnærminger til parsingen. Her bruker vi `pdfminer`, som er en modul i Python som leser font-informasjon i pdf-ene. Vi går da tilbake til pdf-ene for i første omgang å analysere hvilke overskrifter som er overskrifter i vedtakene. En gjennomgang av ulike vedtak gjør at vi kan anta at alle overskrifter i vedtakene er skrevet med uthevet font. Annen tekst bruker ikke uthevet font. Dette må selvfølgelig kvalitetssikres, men det virker som et godt utganspunkt for å identifisere overskrifter.

Hvorfor ikke går rett på de parsed vedtakene og analysere tekstfilene? Det er vanskelig å finne kjennetegn ved overskrifter i rent tekstformat som garanterer at vi identifiserer tekst som overskrift. Det er enkelt å hente ut tekst når vi vet hva overskriften er, og vi antar at dette er likt for alle vedtak. Men før vi kan anta at alle vedtak er like, må vi forsikre oss om at vi vet hva alle overskrifter er, og at alle vedtak har samme struktur.

Strategien er å bruke `pdfminer`til lese ut tekst fra pdf-en. Vi leser ut tekst fra hvert enkelt vedtak, linje for linje. Hvis den første bokstaven i linjen er skrevet med fet font, lagrer vi tekstlinjen som en overskriftskandidat. Vi prøver først å analysere ett enkelt vedtak, her sak 2021-01885.

In [1]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar

def extract_text_with_font(pdf_path):
    text_info = []

    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        # Forsøk å finne den første bokstaven i linjen
                        first_char = next((char for char in text_line if isinstance(char, LTChar)), None)
                        
                        if first_char is not None:
                            # Sjekk om den første bokstaven er uthevet
                            is_bold = "Bold" in first_char.fontname

                            # Hvis den første bokstaven er uthevet, betrakt hele linjen som uthevet
                            if is_bold:
                                text_info.append({
                                    "text": text_line.get_text(),  
                                    "font_name": first_char.fontname,
                                    "size": first_char.size,
                                    "is_bold": is_bold  
                                })

    return text_info

def main(pdf_path):
    teller = 0
    text_info = extract_text_with_font(pdf_path)

    print("\nIdentifiserte tekstlinjer med uthevet tekst:")
    print("--------------------------------------------")
    for info in text_info:
        print(info["text"].strip())  # Dette vil skrive ut hver uthevet linje som en separat streng
        teller +=1
    print("\nTotalt", teller, "overskrifter.")

if __name__ == "__main__":
    pdf_path = "./pdfs_small/2021-01885.pdf"  # erstatt med stien til PDF-filen din
    main(pdf_path)



Identifiserte tekstlinjer med uthevet tekst:
--------------------------------------------
Vedtak i Transportklagenemnda - Fly
Sammendrag
Dato
Saksnummer
Tjenesteytere
Klager har i det vesentlige anført
Tjenesteyterne har i det vesentlige anført
Nemnda bemerker
Vedtak
Nemndas representanter

Totalt 10 overskrifter.


Koden finner det vi forventer. Den har identifisert alle overskrifter og utelatt annen tekst. Dette var for et enkelt vedtak. For å undersøke videre om denne metoden er robust, tester vi på alle vedtak. Det gir ikke mening å printe ut alle overskrifter for 7827 vedtak. Vi leter derfor igjennom tekstlinjer med uthevet font og tar vare på alle unike tekstlinjer vi finner. Det skal da gi alle overskrifter som er brukt i vedtakene. Om metoden plukker med noe mer, vil vi også se det. (Koden under krever ressurser når antallet pdf-er er stort og tar derfor noe tid å kjøre)

In [2]:
import os
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar

def extract_text_with_font(pdf_path):
    text_info = []

    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        # Forsøk å finne den første karakteren i linjen
                        first_char = next((char for char in text_line if isinstance(char, LTChar)), None)

                        if first_char is not None:
                            # Sjekk om den første karakteren er uthevet
                            is_bold = "Bold" in first_char.fontname

                            # Hvis den første karakteren er uthevet, betrakt hele linjen som uthevet
                            if is_bold:
                                text_info.append(text_line.get_text().strip())  # Hent hele linjeteksten

    return text_info

def main(directory_path):
    bold_lines_set = set()

    # Gå gjennom hver fil i katalogen
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            full_path = os.path.join(directory_path, filename)
            # Ekstraher de uthevede linjene fra den nåværende PDF-en og legg dem til i settet
            bold_lines = extract_text_with_font(full_path)
            for line in bold_lines:
                bold_lines_set.add(line)

    print("\nIdentifiserte linjer med uthevet tekst:")
    print("--------------------------------------------")
    for line in bold_lines_set:
        print(line)  # Dette vil skrive ut hver unike uthevede linje som en separat streng

    print("\nTotalt unike linjer med uthevet skrift funnet:", len(bold_lines_set))

if __name__ == "__main__":
    directory_path = "./pdfs/"  # erstatt med din katalogsti
    main(directory_path)




Identifiserte linjer med uthevet tekst:
--------------------------------------------
Tjenesteytere
Tjenesteyter følger ikke vedtaket i saken (23.05.2023)
Tjenesteyter følger ikke vedtaket i saken (02.03.2023)
Tjenesteyter følger ikke vedtaket i saken (28.09.2022)
Tjenesteyter følger ikke vedtaket i saken (12.01.2023)
Tjenesteyter følger ikke vedtaket i saken (09.11.2022)
Nemnda bemerker
- Kollektivreiser
Tjenesteyter følger ikke vedtaket i saken (08.02.2023)
Tjenesteyter følger ikke vedtaket i saken (30.03.2023)
Tjenesteyter følger ikke vedtaket i saken (06.07.2023)
Vedtak
Saksnummer
Sammendrag
Tjenesteyter følger ikke vedtaket i saken (11.10.2022)
Tjenesteyter følger ikke vedtaket i saken (13.01.2021)
Vedtak i Pakkereisenemnda
Tjenesteyter følger ikke vedtaket i saken (20.09.2019)
Vedtak i Transportklagenemnda - Fly
Klager har i det vesentlige anført
Tjenesteyter følger ikke vedtaket i saken (31.01.2023)
Tjenesteyter følger ikke vedtaket i saken (29.11.2022)
Tjenesteyter følger ikke 

Resultatet er betryggende. Vi har analysert 7827 vedtak og sitter igjen med kun 41 forskjellige overskrifter. Det tyder på fast struktur i vedtakene. Mange av disse overskriftene er variasjoner av det samme med et tillegg av en dato. Her kan vi med tanke på parsingen fjerne datoen slik at alle disse blir like. Etter å ha fjernet variasjonen med dato, står vi igjen med 15 ulike overskrifter. Fire av disse er tittel på vedtakene:
- Vedtak i Pakkereisenemnda
- Vedtak i Transportklagenemnda - Fly
- Vedtak i Transportklagenemnda - Sjø
- Vedtak i Transportklagenemnda - Kollektivreiser
I pdf-en er overskriften til Transportklagenemnda - Kollektivreiser delt over to linjer. Derfor kommer den opp både som "Transportklagenemnda" og som "- Kollektivreiser". Fjerner vi også disse titlene, står vi igjen med bare 10 overskrifter til innhold i vedtaket.

For å vite om det er variasjon mellom vedtakene i bruken av overskrifter, dvs. om alle overskrifter er med i alle vedtak, kjører vi en sjekk også på dette. Vi lagrer da alle overskrifter for et vedtak i en liste, så sammenligner vi listene og sjekker hvor mange forskellige lister vi får.

In [3]:
import os
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar
from collections import defaultdict

def extract_text_with_font(pdf_path):
    text_info = []

    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        first_char = next((char for char in text_line if isinstance(char, LTChar)), None)

                        if first_char is not None and "Bold" in first_char.fontname:
                            line_text = text_line.get_text().strip()

                            # Sjekk starten av teksten og anvend forhold
                            if line_text.startswith("Tjenesteyter følger ikke vedtaket i saken"):
                                text_info.append("Tjenesteyter følger ikke vedtaket i saken")
                            elif not (line_text.startswith("Vedtak i") or line_text.startswith("- Kollektivreiser")):
                                text_info.append(line_text)

    return text_info

def main(directory_path):
    all_pdfs_bold_lines = []
    list_occurrences = defaultdict(int)

    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            full_path = os.path.join(directory_path, filename)
            bold_lines = extract_text_with_font(full_path)
            if bold_lines:  # Bare legg til ikke-tomme lister
                # Konverter liste til en tuple slik at den kan hashes for ordboken av forekomster
                bold_lines_tuple = tuple(bold_lines)
                all_pdfs_bold_lines.append(bold_lines_tuple)
                list_occurrences[bold_lines_tuple] += 1

    unique_lists = set(all_pdfs_bold_lines)

    print("\nAntall unike lister: ", len(unique_lists))
    for unique_list in unique_lists:
        print("\nElementer i den unike listen:\n-----------------------")
        for item in unique_list:
            print(item)
        print("\nAntall PDF-er med denne unike listen: ", list_occurrences[unique_list])

if __name__ == "__main__":
    directory_path = "./pdfs" 
    main(directory_path)




Antall unike lister:  2

Elementer i den unike listen:
-----------------------
Sammendrag
Dato
Saksnummer
Tjenesteytere
Klager har i det vesentlige anført
Tjenesteyterne har i det vesentlige anført
Nemnda bemerker
Vedtak
Nemndas representanter

Antall PDF-er med denne unike listen:  7714

Elementer i den unike listen:
-----------------------
Tjenesteyter følger ikke vedtaket i saken
Sammendrag
Dato
Saksnummer
Tjenesteytere
Klager har i det vesentlige anført
Tjenesteyterne har i det vesentlige anført
Nemnda bemerker
Vedtak
Nemndas representanter

Antall PDF-er med denne unike listen:  113


Dette er gode nyheter! Vi finner bare to sett med ulike overskrifter i alle de 7827 vedtakene, og det eneste som skiller dem, er om de er innledet med «Tjenesteyter følger ikke vedtaket i saken». Det gjør det enkelt å parse dokumentene og lagre dem i strukturert form.

## Parsing av vedtak